# Goal reminder 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

### Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python
["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 

# Basic information

First things first: let's load the libraries we will be using, as well as the private data to run our tools such as AWS' and variables to work on.

We used Anaconda as our environment manager. You may already have one to run this notebook but it is highly recommended you create a new environment following these instructions:

1. Run Anaconda prompt
2. `cd` to the working directory where this notebook is located (..\cdsd_kayak)
    * If you cannot remember where you downloaded the project, run `cd (ls C:\Users\ cdsd_kayak -Recurse -Directory).FullName`
    * This simple command assumes it was downloaded on your C:\ drive and will **not** handle exceptions!
3. Install our environment: `conda env create -n M174_kayak --file=kayak_project_environment.yaml`
4. Activate it: `conda activate M174_kayak`
5. Since Anaconda doesn't support METEO France's API, if it failed running the pip command, you may have to install it with `pip install meteofrance-api`
6. Your new environment is ready for use!

If you came back here because of the spiders, here is a [convenient link](#getting-20-hotels-urls-per-location) back to this part. Otherwise, on to the next cell!

In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px

from geopy.geocoders import Nominatim
from meteofrance_api import MeteoFranceClient
from datetime import datetime

import json, requests, os, dotenv, boto3

from sqlalchemy import create_engine, text

In [2]:
holiday_places = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", 
                  "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", 
                  "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", 
                  "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

len(holiday_places)

35

### Environment variables setup

These confidential variables must be set up to run the online parts of this notebook. Please refer to the `.env_example.txt` file for a template & instructions.

For convenience if you were near the end of the notebook, here is a [link back](#pushing-our-final-dataframe-to-s3) to the part that refers the most to these confidential variables.

In [3]:
dotenv.load_dotenv()

HERE_api_key = os.getenv("HERE_api_key")    #requires your registering with a HERE account
HERE_api_url = 'https://geocode.search.hereapi.com/v1/'

AWS_id = os.getenv("AWS_ACCESS_KEY_ID")     #all of these will be found under your AWS "IAM" service
AWS_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_bucket = os.getenv("S3_BUCKET_NAME")
S3_uri = os.getenv("S3_BUCKET_URI")

neonDB = os.getenv("SQL_NEONDB_URI")        #finally, this one requires setting up your own account & neonDB project

# Getting GPS coordinates

We will first initialize Nominatim, while raising its timeout value since it can take a few seconds to return a result.

Then we'll iterate on our places' coordinates:

In [4]:
geolocator = Nominatim(timeout=10, user_agent="weather_app")

In [32]:
for place in holiday_places:
    location = geolocator.geocode(f'{place}, France')
    if location:
        print(f"{place} → lat: {location.latitude}, lon: {location.longitude}")
    else:
        print(f"Unable to find {place}")

Mont Saint Michel → lat: 48.6359541, lon: -1.51146
St Malo → lat: 48.649518, lon: -2.0260409
Bayeux → lat: 49.2764624, lon: -0.7024738
Le Havre → lat: 49.4938975, lon: 0.1079732
Rouen → lat: 49.4404591, lon: 1.0939658
Paris → lat: 48.8588897, lon: 2.320041
Amiens → lat: 49.8941708, lon: 2.2956951
Lille → lat: 50.6365654, lon: 3.0635282
Strasbourg → lat: 48.584614, lon: 7.7507127
Chateau du Haut Koenigsbourg → lat: 48.2494107, lon: 7.3443202
Colmar → lat: 48.0777517, lon: 7.3579641
Eguisheim → lat: 48.0447968, lon: 7.3079618
Besancon → lat: 47.2380222, lon: 6.0243622
Dijon → lat: 47.3215806, lon: 5.0414701
Annecy → lat: 45.8992348, lon: 6.1288847
Grenoble → lat: 45.1875602, lon: 5.7357819
Lyon → lat: 45.7578137, lon: 4.8320114
Gorges du Verdon → lat: 43.7496562, lon: 6.3285616
Bormes les Mimosas → lat: 43.1506968, lon: 6.3419285
Cassis → lat: 43.2140359, lon: 5.5396318
Marseille → lat: 43.2961743, lon: 5.3699525
Aix en Provence → lat: 43.5298424, lon: 5.4474738
Avignon → lat: 43.9492493

# Getting weather data

Next for the weather data, we'll first run a little test using the French weather service Meteo France:

In [5]:
client = MeteoFranceClient()

In [6]:
def get_forecast_by_city(city_name):
    location = geolocator.geocode(f'{city_name}, France') #recovering Nominatim's GPS coordinates
    if not location:
        return city_name, None

    forecast = client.get_forecast(location.latitude, location.longitude)
    
    daily = forecast.daily_forecast[:2] #forecast recovery for the next two days
    id_place = forecast.position['insee'] #unique id based on France's national institute for statistics
    return city_name, daily, id_place

In [35]:
for place in holiday_places:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"Forecast for {city} :")
        print(f'INSEE id {id_place} \n')
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            print(f" - {date}: {tmin}°C → {tmax}°C")
        print()
    else:
        print(f"!!! Error encountered for {city} !!!")

Forecast for Mont Saint Michel, France :
INSEE id 5035351 

→ lat: 48.6359541, lon: -1.51146 

 - 2025-11-13: 12.5°C → 17.2°C
 - 2025-11-14: 13.4°C → 18.2°C

Forecast for St Malo, France :
INSEE id 3528885 

→ lat: 48.649518, lon: -2.0260409 

 - 2025-11-13: 13.6°C → 17.4°C
 - 2025-11-14: 14.6°C → 18.7°C

Forecast for Bayeux, France :
INSEE id 140470 

→ lat: 49.2764624, lon: -0.7024738 

 - 2025-11-13: 13.4°C → 18.1°C
 - 2025-11-14: 13.9°C → 19°C

Forecast for Le Havre, France :
INSEE id 763510 

→ lat: 49.4938975, lon: 0.1079732 

 - 2025-11-13: 13.1°C → 18.1°C
 - 2025-11-14: 12.5°C → 19.5°C

Forecast for Rouen, France :
INSEE id 765400 

→ lat: 49.4404591, lon: 1.0939658 

 - 2025-11-13: 9.9°C → 17.7°C
 - 2025-11-14: 11.4°C → 19.6°C

Forecast for Paris, France :
INSEE id 751070 

→ lat: 48.8588897, lon: 2.320041 

 - 2025-11-13: 11.5°C → 17.7°C
 - 2025-11-14: 13.2°C → 20.1°C

Forecast for Amiens, France :
INSEE id 800210 

→ lat: 49.8941708, lon: 2.2956951 

 - 2025-11-13: 12.2°C → 

In [36]:
daily_forecast

[{'dt': 1762992000,
  'T': {'min': 14.1, 'max': 20.5, 'sea': None},
  'humidity': {'min': 65, 'max': 85},
  'precipitation': {'24h': 0},
  'uv': 1,
  'weather12H': {'icon': 'p3bisj', 'desc': 'Couvert'},
  'sun': {'rise': 1763017323, 'set': 1763051706}},
 {'dt': 1763078400,
  'T': {'min': 15.4, 'max': 19.1, 'sea': None},
  'humidity': {'min': 70, 'max': 90},
  'precipitation': {'24h': 1.1},
  'uv': 1,
  'weather12H': {'icon': 'p12j', 'desc': 'Pluie faible'},
  'sun': {'rise': 1763103808, 'set': 1763138039}}]

The variable printed above returns a lot of interesting informations we may use for our objective. Let's extract those:

In [37]:
for place in holiday_places:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"=== Forecast for {city} :")
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            rainfall = day['precipitation']['24h']
            uv = day['uv']
            weather = day['weather12H']['desc']
            id = day['weather12H']['desc']
            print(f" - {date}: {tmin}°C → {tmax}°C \n rainfall => {rainfall} uv => {uv} weather => {weather}")
        print()
    else:
        print(f"!!! Error encountered for {city} !!!")

=== Forecast for Mont Saint Michel, France :
→ lat: 48.6359541, lon: -1.51146 

 - 2025-11-13: 12.5°C → 17.2°C 
 rainfall => 0.6 uv => 1 weather => Pluie faible
 - 2025-11-14: 13.4°C → 18.2°C 
 rainfall => 2.3 uv => 1 weather => Pluie faible

=== Forecast for St Malo, France :
→ lat: 48.649518, lon: -2.0260409 

 - 2025-11-13: 13.6°C → 17.4°C 
 rainfall => 0.4 uv => 1 weather => Pluie faible
 - 2025-11-14: 14.6°C → 18.7°C 
 rainfall => 2.8 uv => 1 weather => Pluie faible

=== Forecast for Bayeux, France :
→ lat: 49.2764624, lon: -0.7024738 

 - 2025-11-13: 13.4°C → 18.1°C 
 rainfall => 0.1 uv => 1 weather => Couvert
 - 2025-11-14: 13.9°C → 19°C 
 rainfall => 0.1 uv => 1 weather => Couvert

=== Forecast for Le Havre, France :
→ lat: 49.4938975, lon: 0.1079732 

 - 2025-11-13: 13.1°C → 18.1°C 
 rainfall => 0.5 uv => 1 weather => Pluie faible
 - 2025-11-14: 12.5°C → 19.5°C 
 rainfall => 0.6 uv => 1 weather => Pluie faible

=== Forecast for Rouen, France :
→ lat: 49.4404591, lon: 1.0939658

So far so good, so let's produce now our weather forecast for one week ahead and store the information into a Pandas dataframe:

In [7]:
def get_forecast_by_city_fordf(city_name):
    location = geolocator.geocode(f'{city_name}, France')
    if not location:
        return city_name, None

    forecast = client.get_forecast(location.latitude, location.longitude)
    
    daily = forecast.daily_forecast[:7] #value set for the whole week
    id_place = forecast.position['insee']
    return city_name, daily, id_place, location.latitude, location.longitude

In [13]:
data_weather = []
for place in holiday_places:
    city, daily_forecast, id_place, lat, lon = get_forecast_by_city_fordf(place)
    if daily_forecast:
        for day in daily_forecast:
            row = {
                "insee": id_place,                
                "place": city,
                "lat": lat,
                "lon": lon,
                "date": datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d"),
                "tmin": day["T"]["min"],
                "tmax": day["T"]["max"],
                "rainfall": day['precipitation']['24h']
            }
            data_weather.append(row)

df_weather = pd.DataFrame(data_weather)
df_weather

,insee,place,lat,lon,date,tmin,tmax,rainfall
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-13,12.5,17.6,0.1
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-14,12.9,18.6,1.0
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-15,10.4,14.6,4.4
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-16,10.0,13.2,0.0
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-17,7.0,12.2,0.0
...,...,...,...,...,...,...,...,...
240,173000,La Rochelle,46.159732,-1.151595,2025-11-15,13.4,17.6,1.2
241,173000,La Rochelle,46.159732,-1.151595,2025-11-16,11.2,16.9,0.0
242,173000,La Rochelle,46.159732,-1.151595,2025-11-17,9.2,13.6,0.0
243,173000,La Rochelle,46.159732,-1.151595,2025-11-18,4.9,13.2,0.0


In [ ]:
pd.DataFrame.to_csv(df_weather, "data/weekly_weather_forecast.csv", index=False)

We now have weekly forecasts! The previous cell allowed us to store it into a csv file for later retrieval - it also serves as a checkpoint for the next steps.

### Selecting weather data

We will now work on our forecasts to enable the display of the average temperature on a given day, the expected rainfall, then the weekly average temperature:

In [16]:
df_weather = pd.read_csv("data/weekly_weather_forecast.csv")
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   insee     245 non-null    int64  
 1   place     245 non-null    object 
 2   lat       245 non-null    float64
 3   lon       245 non-null    float64
 4   date      245 non-null    object 
 5   tmin      245 non-null    float64
 6   tmax      245 non-null    float64
 7   rainfall  245 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 15.4+ KB


In [17]:
df_weather.head()

,insee,place,lat,lon,date,tmin,tmax,rainfall
0,5035351,Mont Saint Michel,48.635954,-1.51146,2025-11-13,12.5,17.6,0.1
1,5035351,Mont Saint Michel,48.635954,-1.51146,2025-11-14,12.9,18.6,1.0
2,5035351,Mont Saint Michel,48.635954,-1.51146,2025-11-15,10.4,14.6,4.4
3,5035351,Mont Saint Michel,48.635954,-1.51146,2025-11-16,10.0,13.2,0.0
4,5035351,Mont Saint Michel,48.635954,-1.51146,2025-11-17,7.0,12.2,0.0


In [8]:
df_weather['td_mean'] = np.mean(df_weather[['tmin', 'tmax']], axis=1) #daily average temperature
df_weather

,insee,place,lat,lon,date,tmin,tmax,rainfall,td_mean
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-13,12.5,17.6,0.1,15.05
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-14,12.9,18.6,1.0,15.75
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-15,10.4,14.6,4.4,12.50
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-16,10.0,13.2,0.0,11.60
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-17,7.0,12.2,0.0,9.60
...,...,...,...,...,...,...,...,...,...
240,173000,La Rochelle,46.159732,-1.151595,2025-11-15,13.4,17.6,1.2,15.50
241,173000,La Rochelle,46.159732,-1.151595,2025-11-16,11.2,16.9,0.0,14.05
242,173000,La Rochelle,46.159732,-1.151595,2025-11-17,9.2,13.6,0.0,11.40
243,173000,La Rochelle,46.159732,-1.151595,2025-11-18,4.9,13.2,0.0,9.05


In [9]:
df_w = df_weather.copy(deep=True)
df_w['rainw_mean'] = df_w.groupby('insee')['rainfall'].transform("mean") #weekly average rainfall
df_w['tw_mean'] = df_w.groupby('insee')['td_mean'].transform("mean") #weekly average temperature
df_w.head(10)

,insee,place,lat,lon,date,tmin,tmax,rainfall,td_mean,rainw_mean,tw_mean
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-13,12.5,17.6,0.1,15.05,2.671429,11.271429
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-14,12.9,18.6,1.0,15.75,2.671429,11.271429
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-15,10.4,14.6,4.4,12.50,2.671429,11.271429
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-16,10.0,13.2,0.0,11.60,2.671429,11.271429
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-17,7.0,12.2,0.0,9.60,2.671429,11.271429
5,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-18,3.9,10.4,3.0,7.15,2.671429,11.271429
6,5035351,Mont Saint Michel,48.635954,-1.511460,2025-11-19,5.4,9.1,10.2,7.25,2.671429,11.271429
7,3528885,St Malo,48.649518,-2.026041,2025-11-13,12.9,18.1,0.7,15.50,2.900000,12.314286
8,3528885,St Malo,48.649518,-2.026041,2025-11-14,13.7,19.1,1.4,16.40,2.900000,12.314286
9,3528885,St Malo,48.649518,-2.026041,2025-11-15,10.5,16.2,2.6,13.35,2.900000,12.314286


With our daily & weekly averages now available, let's bring it all back to a single row per location:

In [10]:
df_w1 = df_w.groupby(by='insee').last().reset_index()
df_w1.head()

,insee,place,lat,lon,date,tmin,tmax,rainfall,td_mean,rainw_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-11-19,-1.9,11.1,0.2,4.60,6.357143,8.378571
1,91960,Ariege,42.945537,1.406554,2025-11-19,2.6,9.0,0.6,5.80,1.571429,10.828571
2,110690,Carcassonne,43.213036,2.349107,2025-11-19,4.6,12.7,0.0,8.65,0.028571,14.371429
3,130010,Aix en Provence,43.529842,5.447474,2025-11-19,5.6,13.1,0.0,9.35,6.528571,13.850000
4,130220,Cassis,43.214036,5.539632,2025-11-19,9.2,15.4,0.0,12.30,8.857143,16.121429


Then we will keep the key informations to determine what we consider a "nice weather" for holidays. For simplicity, we'll keep weekly rainfall & temperatures:

In [11]:
df_w2 = df_w1.drop(columns=['tmin', 'tmax', 'rainfall', 'td_mean'])
df_w2.head()

,insee,place,lat,lon,date,rainw_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-11-19,6.357143,8.378571
1,91960,Ariege,42.945537,1.406554,2025-11-19,1.571429,10.828571
2,110690,Carcassonne,43.213036,2.349107,2025-11-19,0.028571,14.371429
3,130010,Aix en Provence,43.529842,5.447474,2025-11-19,6.528571,13.850000
4,130220,Cassis,43.214036,5.539632,2025-11-19,8.857143,16.121429


Then we will begin by determining a "nice weather" as one with little rainfalls:

In [12]:
df_w3 = df_w2[df_w2['rainw_mean'] <= 3 ]
df_w3.head(20)

,insee,place,lat,lon,date,rainw_mean,tw_mean
1,91960,Ariege,42.945537,1.406554,2025-11-19,1.571429,10.828571
2,110690,Carcassonne,43.213036,2.349107,2025-11-19,0.028571,14.371429
6,140470,Bayeux,49.276462,-0.702474,2025-11-19,1.871429,11.178571
7,173000,La Rochelle,46.159732,-1.151595,2025-11-19,1.257143,13.521429
13,315550,Toulouse,43.604464,1.444243,2025-11-19,0.157143,13.992857
15,593500,Lille,50.636565,3.063528,2025-11-19,0.785714,9.600000
17,660530,Collioure,42.525050,3.083155,2025-11-19,0.271429,16.178571
18,674820,Strasbourg,48.584614,7.750713,2025-11-19,1.914286,8.942857
19,680660,Colmar,48.077752,7.357964,2025-11-19,2.357143,9.107143
20,680780,Eguisheim,48.044797,7.307962,2025-11-19,2.342857,9.000000


Next, we will keep the highest temperatures available given the season - for the sake of summer, it would take a little more work to remain below 35°C:

In [13]:
df_w4 = df_w3.sort_values(by='tw_mean', ascending=False)

And finally, we can extract our top 5 locations!

In [14]:
df_w5 = df_w4.head(5)
df_w5

,insee,place,lat,lon,date,rainw_mean,tw_mean
17,660530,Collioure,42.525050,3.083155,2025-11-19,0.271429,16.178571
31,1309651,Saintes Maries de la mer,43.451592,4.427720,2025-11-19,2.371429,15.335714
2,110690,Carcassonne,43.213036,2.349107,2025-11-19,0.028571,14.371429
13,315550,Toulouse,43.604464,1.444243,2025-11-19,0.157143,13.992857
28,821210,Montauban,44.017584,1.354999,2025-11-19,0.200000,13.628571


# Top5 locations: first visualization

Now that we have a top 5 for our locations with the best weather available, let's have them displayed on a map to get an idea of the end result we expect!

In [ ]:
fig5 = px.scatter_map(df_w5, lat="lat", lon="lon", color="tw_mean", hover_name="place", 
                     hover_data=["rainw_mean", "tw_mean"], size="tw_mean", zoom=4)
fig5.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig5.show()

# Scrapping hotels' informations

Our weather data per location is ready, we can deal with the hotels now. Follow the next steps to begin the scrapping part:

### Getting 20 hotels' urls per location

1. Be sure you followed our [earlier instructions](#basic-information) in setting up your environment, and running this notebook with the new M174_kayak.
2. Run our first spider: `python src/booking_url_hotel.py`
    * Should it fail to execute, check again what path is displayed before your commands - it **must** be the cdsd_kayak folder to work as intended.
3. Keep your Anaconda prompt open until the spider is done (see below) - we will execute another one in the same context right after!
    * Feel free to check the json output under `data/all_cities_url.json` to ensure all 700 rows are available without any missing values.

### Recovering each hotel's data

1. Make sure the previous spider's work successfully finished; its last message should end with `INFO: Spider closed (finished)`.
    * In any other case, you will likely have to repeat the previous spider's steps - especially making sure you're executing commands in the right working directory.
    * While working on the project, data traffic seems to hamper our data recovery. You may have to rerun the spider.
2. Since you should still be under said working directory, run our second spider with `python src/booking_info_hotel.py`
3. Give it again time to complete its work (~45 minutes), this one will take longer and wait for the `INFO: Spider closed (finished)` message!
    * Feel free again to check the json output under `data/hotels_details.json` to make sure at least all informations were recovered on at least one hotel.
    * This part is the most likely to fail with data traffic, keep trying! `ctrl+C` in your terminal will interrupt the spider if you can tell it is failing.

### Merging our informations

With our data now extracted, we can now push further by merging it all into a cohesive json file!

In [16]:
df_w2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   insee       35 non-null     int64  
 1   place       35 non-null     object 
 2   lat         35 non-null     float64
 3   lon         35 non-null     float64
 4   date        35 non-null     object 
 5   rainw_mean  35 non-null     float64
 6   tw_mean     35 non-null     float64
dtypes: float64(4), int64(1), object(2)
memory usage: 2.0+ KB


In [17]:
df_w2.head()

,insee,place,lat,lon,date,rainw_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-11-19,6.357143,8.378571
1,91960,Ariege,42.945537,1.406554,2025-11-19,1.571429,10.828571
2,110690,Carcassonne,43.213036,2.349107,2025-11-19,0.028571,14.371429
3,130010,Aix en Provence,43.529842,5.447474,2025-11-19,6.528571,13.850000
4,130220,Cassis,43.214036,5.539632,2025-11-19,8.857143,16.121429


In [18]:
with open("data/hotels_details.json", "r", encoding="utf-8") as f:
    hotels = json.load(f)

df_hotels = pd.DataFrame(hotels)
df_merged = pd.merge(df_hotels, df_w2, on='place', how="right")
hotels_enriched = df_merged.to_dict(orient="records") #associating columns & data as keys & values for JSON dumping

with open("data/all_hotels_details_insee.json", "w", encoding="utf-8") as f:
    json.dump(hotels_enriched, f, indent=4, ensure_ascii=False) #save as JSON for later use

# Finalizing the transformations of our data

Now that data collection is done, let's give it its final shape! First on the list is cleaning our dataset for ease of use:

In [9]:
with open("data/all_hotels_details_insee.json", "r", encoding="utf-8") as f:
    all_hotels_details = json.load(f)
df = pd.DataFrame(all_hotels_details)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   place        700 non-null    object 
 1   url          700 non-null    object 
 2   name         700 non-null    object 
 3   note         700 non-null    object 
 4   address      700 non-null    object 
 5   description  700 non-null    object 
 6   insee        700 non-null    int64  
 7   lat          700 non-null    float64
 8   lon          700 non-null    float64
 9   date         700 non-null    object 
 10  rainw_mean   700 non-null    float64
 11  tw_mean      700 non-null    float64
dtypes: float64(4), int64(1), object(7)
memory usage: 65.8+ KB


In [10]:
df.head(1)

,place,url,name,note,address,description,insee,lat,lon,date,rainw_mean,tw_mean
0,Gorges du Verdon,https://www.booking.com/hotel/fr/studio-arc-en...,L'Arc en Ciel,Avec une note de 9.1,"80 Chemin du Plan, 04800 Gréoux-les-Bains, France",L’établissement L'Arc en Ciel se situe à 17 km...,41440,43.749656,6.328562,2025-11-19,6.357143,8.378571


We'd like a convenient note format, so a little work on it is required:

In [11]:
df['note'] = df['note'].str.split('de ').str[-1]
df['note'] = pd.to_numeric(df['note'], errors="coerce")
df.head(1)

,place,url,name,note,address,description,insee,lat,lon,date,rainw_mean,tw_mean
0,Gorges du Verdon,https://www.booking.com/hotel/fr/studio-arc-en...,L'Arc en Ciel,9.1,"80 Chemin du Plan, 04800 Gréoux-les-Bains, France",L’établissement L'Arc en Ciel se situe à 17 km...,41440,43.749656,6.328562,2025-11-19,6.357143,8.378571


Now we will want to recover GPS coordinates for each precise address, rather than per city/area. We've chosen HERE for this purpose for its free tier:

In [ ]:
def geocode_here(address):
    r = requests.get(f"{HERE_api_url}geocode?q={address}&apikey={HERE_api_key}").json()
    #html response is in binary format; to json for ease of items' retrieval
    if "items" in r and len(r["items"]) > 0:
        lat_h = r["items"][0]["position"]["lat"]
        lon_h = r["items"][0]["position"]["lng"]
        return lat_h, lon_h
    else:
        return None, None #exception management

In [ ]:
# Uncomment the line below to extract precise GPS coordinates; remember to put it back as a comment afterwards to avoid requesting your HERE API key too often, possibly incurring fees after a while!
#df[["lat_h", "lon_h"]] = df["address"].apply(lambda x: pd.Series(geocode_here(x)))
df.head(1)

,place,url,name,note,address,description,insee,lat,lon,date,rainw_mean,tw_mean,lat_h,lon_h
0,Gorges du Verdon,https://www.booking.com/hotel/fr/studio-arc-en...,L'Arc en Ciel,9.1,"80 Chemin du Plan, 04800 Gréoux-les-Bains, France",L’établissement L'Arc en Ciel se situe à 17 km...,41440,43.749656,6.328562,2025-11-19,6.357143,8.378571,43.75664,5.87664


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   place        700 non-null    object 
 1   url          700 non-null    object 
 2   name         700 non-null    object 
 3   note         700 non-null    float64
 4   address      700 non-null    object 
 5   description  700 non-null    object 
 6   insee        700 non-null    int64  
 7   lat          700 non-null    float64
 8   lon          700 non-null    float64
 9   date         700 non-null    object 
 10  rainw_mean   700 non-null    float64
 11  tw_mean      700 non-null    float64
 12  lat_h        700 non-null    float64
 13  lon_h        700 non-null    float64
dtypes: float64(7), int64(1), object(6)
memory usage: 76.7+ KB


# Top20 hotels per location

We finally have the proper state for our dataframe to reach our goals! Even if we expect some cluttering, how about a preview of the final result?

### Best hotels by user notes

In [ ]:
fig20n = px.scatter_map(df, lat="lat_h", lon="lon_h", color="note", color_continuous_scale="Turbo",
                     hover_name="name", hover_data=["url", "address"], zoom=4)
fig20n.update_traces(marker_size=10)
fig20n.update_layout(map_style="basic", margin={"r":0,"t":0,"l":0,"b":0})
fig20n.show()

### Best hotels by expected temperature

In [ ]:
fig20t = px.scatter_map(df, lat="lat", lon="lon", color="tw_mean", color_continuous_scale="Turbo",
                     hover_name="name", hover_data=["url", "address"], zoom=4)
fig20t.update_traces(marker_size=10)
fig20t.update_layout(map_style="basic", margin={"r":0,"t":0,"l":0,"b":0})
fig20t.show()

In [68]:
pd.DataFrame.to_csv(df, "data/hotels_info.csv", index=False)

# Building our online infrastructure

Since we will be referring to environment variables defined at the very beginning, you may want to jump back there with [this link](#environment-variables-setup) for a reminder!

### Pushing our final dataframe to S3

In [8]:
session = boto3.Session(aws_access_key_id=AWS_id, aws_secret_access_key=AWS_secret_key)
s3 = boto3.client('s3')

In [ ]:
# Uncomment the next line to actually push the file; don't forget to comment it back to avoid multiple pushes that may be billed by AWS!
#s3.upload_file("data/hotels_info.csv", S3_bucket, "kayak_project/hotels_info.csv")

### Connecting to our distant SQL

In [9]:
engine = create_engine(neonDB, echo=True)

In [ ]:
# Uncomment the following lines to retrieve our newly created S3 file before pushing it to the SQL database; once more to avoid billing (by NeonDB), be sure to comment it back after the push!
#df_transit = pd.read_csv(f"{S3_uri}hotels_info.csv")
#df_transit.to_sql("kayak_project", engine, if_exists="replace", index=True)

2025-11-18 20:25:42,543 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-11-18 20:25:42,544 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 20:25:42,599 INFO sqlalchemy.engine.Engine select current_schema()
2025-11-18 20:25:42,599 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 20:25:42,653 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-11-18 20:25:42,654 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 20:25:42,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-18 20:25:42,714 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

700

Then a little preview to make sure everything works as intended?

In [26]:
test_statement = text("SELECT * FROM kayak_project ORDER BY index")
test_conn = engine.connect()
test_result = test_conn.execute(test_statement)
test_result.fetchall()

2025-11-18 23:14:09,487 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-11-18 23:14:09,488 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 23:14:09,540 INFO sqlalchemy.engine.Engine select current_schema()
2025-11-18 23:14:09,541 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 23:14:09,594 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-11-18 23:14:09,594 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-18 23:14:09,647 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-18 23:14:09,649 INFO sqlalchemy.engine.Engine SELECT * FROM kayak_project ORDER BY index
2025-11-18 23:14:09,650 INFO sqlalchemy.engine.Engine [generated in 0.00249s] {}


[(0, 'Gorges du Verdon', 'https://www.booking.com/hotel/fr/studio-arc-en-ciel.fr.html', "L'Arc en Ciel", 9.1, '80 Chemin du Plan, 04800 Gréoux-les-Bains, France', "L’établissement L'Arc en Ciel se situe à 17 km de ce lieu d’intérêt : Agence ITER de Cadarache. Il possède une connexion Wi-Fi gratuite et un parking ... (486 characters truncated) ... s lui donnent la note de   pour un séjour à deux. Les distances indiquées dans la description de l'établissement sont calculées avec © OpenStreetMap.", 41440, 43.7496562, 6.3285616, '2025-11-19', 6.357142857142857, 8.378571428571428, 43.75664, 5.87664),
 (1, 'Gorges du Verdon', 'https://www.booking.com/hotel/fr/du-vieux-chateau.fr.html', 'Le Vieux Château', 7.5, 'Place de la Fontaine, 83630 Aiguines, France', "Occupant un bâtiment du XVIIIe siècle, Le Vieux Château propose des chambres avec connexion Wi-Fi gratuite, un bar et une terrasse. Situé dans le cen ... (797 characters truncated) ... s lui donnent la note de   pour un séjour à deux. Le

# ETL wrap-up

With our various services now connected and the extraction part of our ETL process working as expected, time to make the magic work with some SQL alchemy!

### Top5 locations

In [30]:
top5_statement = text("SELECT DISTINCT(place), lat, lon, tw_mean, rainw_mean FROM kayak_project WHERE rainw_mean <= 3 ORDER BY tw_mean DESC LIMIT 5")
top5_connect = engine.connect()
top5_result = top5_connect.execute(top5_statement)
top5_result.fetchall()

2025-11-19 11:19:05,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-19 11:19:05,844 INFO sqlalchemy.engine.Engine SELECT DISTINCT(place), lat, lon, tw_mean, rainw_mean FROM kayak_project WHERE rainw_mean <= 3 ORDER BY tw_mean DESC LIMIT 5
2025-11-19 11:19:05,844 INFO sqlalchemy.engine.Engine [cached since 1825s ago] {}


[('Collioure', 42.52505, 3.0831554, 16.178571428571427, 0.2714285714285714),
 ('Saintes Maries de la mer', 43.4515922, 4.4277202, 15.335714285714284, 2.3714285714285714),
 ('Carcassonne', 43.2130358, 2.3491069, 14.37142857142857, 0.0285714285714285),
 ('Toulouse', 43.6044638, 1.4442433, 13.992857142857144, 0.1571428571428571),
 ('Montauban', 44.0175835, 1.3549991, 13.62857142857143, 0.1999999999999999)]

In [ ]:
df_top5 = pd.read_sql_query(sql=top5_statement, con=top5_connect)
fig_top5 = px.scatter_map(df_top5, lat="lat", lon="lon", color="tw_mean", hover_name="place", 
                     hover_data=["rainw_mean", "tw_mean"], size="tw_mean", zoom=4)
fig_top5.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_top5.show()

2025-11-19 11:19:19,766 INFO sqlalchemy.engine.Engine SELECT DISTINCT(place), lat, lon, tw_mean, rainw_mean FROM kayak_project WHERE rainw_mean <= 3 ORDER BY tw_mean DESC LIMIT 5
2025-11-19 11:19:19,767 INFO sqlalchemy.engine.Engine [cached since 1839s ago] {}


### Top20 hotels

Since we're using the average weekly temperature for sorting, we can conveniently extract all hotels from the warmest city with a simple LIMIT 20 clause:

In [36]:
top20_statement = text("SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean FROM (" \
"SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean " \
"FROM kayak_project WHERE rainw_mean <= 3) ORDER BY tw_mean DESC LIMIT 20")
top20_connect = engine.connect()
top20_result = top20_connect.execute(top20_statement)
top20_result.fetchall()

2025-11-19 11:48:21,770 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-19 11:48:21,771 INFO sqlalchemy.engine.Engine SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean FROM (SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean FROM kayak_project WHERE rainw_mean <= 3) ORDER BY tw_mean DESC LIMIT 20
2025-11-19 11:48:21,772 INFO sqlalchemy.engine.Engine [generated in 0.00248s] {}


[('66B - Beautiful flat 800m from the sea with private parking', 'https://www.booking.com/hotel/fr/bel-appartement-a-800-m-de-la-mer-parking-prive.fr.html', '27 Rue Romain Rolland, 66190 Collioure, France', 8.9, 42.52803, 3.07775, 0.2714285714285714, 16.178571428571427),
 ('Le Conflent WIFI Terrace', 'https://www.booking.com/hotel/fr/le-conflent-wifi-terrace.fr.html', '34 Rue du Soleil, 66190 Collioure, France', 8.1, 42.52323, 3.08461, 0.2714285714285714, 16.178571428571427),
 ('Hôtel Princes de Catalogne', 'https://www.booking.com/hotel/fr/princes-de-catalogne.fr.html', 'Rue Des Palmiers, 66190 Collioure, France', 8.4, 42.52616, 3.0807, 0.2714285714285714, 16.178571428571427),
 ('Les Suites de Collioure', 'https://www.booking.com/hotel/fr/les-suites-de-collioure.fr.html', '16 avenue du Général de Gaulle, 66190 Collioure, France', 9.3, 42.5244, 3.08273, 0.2714285714285714, 16.178571428571427),
 ('Résidence St Vincent', 'https://www.booking.com/hotel/fr/residence-saint-vincent.fr.html',

In [43]:
df_top20 = pd.read_sql_query(sql=top20_statement, con=top20_connect)
fig_top20 = px.scatter_map(df_top20, lat="lat_h", lon="lon_h", color="note", color_continuous_scale="Turbo",
                     hover_name="name", hover_data=["url", "address"], zoom=14)
fig_top20.update_traces(marker_size=10)
fig_top20.update_layout(map_style="basic", margin={"r":0,"t":0,"l":0,"b":0})
fig_top20.show()

2025-11-19 11:51:13,081 INFO sqlalchemy.engine.Engine SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean FROM (SELECT name, url, address, note, lat_h, lon_h, rainw_mean, tw_mean FROM kayak_project WHERE rainw_mean <= 3) ORDER BY tw_mean DESC LIMIT 20
2025-11-19 11:51:13,081 INFO sqlalchemy.engine.Engine [cached since 171.3s ago] {}
